# Silent Hunter v6.0 - ISS Metagenome Novel Protein Discovery

**100% Data Publication Pipeline**

## FAHDYCONX RULES
1. **NO FABRICATION** - Every number from actual output
2. **DOCUMENT TRANSITIONS** - Record INPUT → OUTPUT for each step
3. **READ DOCS FIRST** - Understand tools before running

## Your Files (Already on Drive)
- `SilentHunter_v6/SRR6356483` - Raw ISS data (4.5 GB)
- `SilentHunter_v6/uniref90.fasta.gz` - Database (35 GB)

## Cell 0: Setup and Mount Drive

In [ ]:
# Mount Google Drive and setup logging
from google.colab import drive
import datetime
import hashlib
import os
import json

drive.mount('/content/drive')

# Base directory - your files are here
BASE_DIR = '/content/drive/MyDrive/SilentHunter_v6'

# Create subdirectories
for d in ['intermediate', 'output', 'audit']:
    os.makedirs(f'{BASE_DIR}/{d}', exist_ok=True)

# Verify your files exist
SRA_FILE = f'{BASE_DIR}/SRR6356483'
UNIREF90_FASTA = f'{BASE_DIR}/uniref90.fasta.gz'

print("Checking your files...")
print(f"SRA file: {os.path.exists(SRA_FILE)} - {SRA_FILE}")
print(f"UniRef90: {os.path.exists(UNIREF90_FASTA)} - {UNIREF90_FASTA}")

if not os.path.exists(SRA_FILE):
    print("\n⚠️ ERROR: SRR6356483 not found! Upload it to SilentHunter_v6/")
if not os.path.exists(UNIREF90_FASTA):
    print("\n⚠️ ERROR: uniref90.fasta.gz not found!")

# Configuration
SRA_ACCESSION = 'SRR6356483'
THREADS = 4

# Audit logging
AUDIT_LOG = f'{BASE_DIR}/audit/commands.log'

def log_command(cmd, output=""):
    with open(AUDIT_LOG, 'a') as f:
        f.write(f"\n{'='*60}\n")
        f.write(f"TIME: {datetime.datetime.now().isoformat()}\n")
        f.write(f"CMD: {cmd}\n")
        if output:
            f.write(f"OUTPUT: {output[:500]}...\n")
    print(f"[LOGGED] {cmd[:50]}...")

log_command("Session started")
print(f"\n✅ Base directory: {BASE_DIR}")
print("✅ Audit logging enabled")

## Step 1: Convert SRA to FASTQ (Using Your Downloaded File)

In [ ]:
# Install SRA Toolkit
!pip install -q sra-tools

# Copy SRA file to working directory (faster than reading from Drive)
print("Copying SRA file to local storage...")
!cp "{BASE_DIR}/SRR6356483" .
!ls -lh SRR6356483

# Convert SRA to FASTQ
print("\nConverting SRA to FASTQ (this takes ~10-15 minutes)...")
cmd = "fastq-dump --split-files --gzip SRR6356483"
!{cmd}
log_command(cmd)

# Verify conversion
print("\n" + "="*60)
print("STEP 1 COMPLETE: SRA → FASTQ")
print("="*60)
!ls -lh SRR6356483*.fastq.gz

# Count reads
read_count = !zcat SRR6356483_1.fastq.gz | wc -l
num_reads = int(read_count[0]) // 4
print(f"\nTotal reads: {num_reads:,}")

# Calculate checksums
!md5sum SRR6356483*.fastq.gz > {BASE_DIR}/audit/raw_data.md5

print(f"\n📊 TRANSITION LOG:")
print(f"INPUT:  SRR6356483 (SRA format)")
print(f"OUTPUT: SRR6356483_1.fastq.gz, SRR6356483_2.fastq.gz")
print(f"READS:  {num_reads:,} paired-end reads")

## Step 2: Quality Control (fastp)

In [ ]:
# Install fastp
!apt-get install -qq fastp

print("Running fastp quality control...")
print("Parameters:")
print("  --cut_mean_quality 20: Remove bases below Q20")
print("  --length_required 50: Remove reads < 50bp")
print()

# Run fastp
cmd = f"""fastp \
    -i SRR6356483_1.fastq.gz \
    -I SRR6356483_2.fastq.gz \
    -o clean_1.fastq.gz \
    -O clean_2.fastq.gz \
    --detect_adapter_for_pe \
    --cut_front --cut_tail \
    --cut_window_size 4 \
    --cut_mean_quality 20 \
    --length_required 50 \
    --html {BASE_DIR}/audit/fastp_report.html \
    --json {BASE_DIR}/audit/fastp_report.json \
    --thread {THREADS}"""

!{cmd}
log_command(cmd)

# Display QC stats
with open(f'{BASE_DIR}/audit/fastp_report.json') as f:
    data = json.load(f)
    before = data['summary']['before_filtering']
    after = data['summary']['after_filtering']

print("\n" + "="*60)
print("STEP 2 COMPLETE: Quality Control")
print("="*60)
print(f"\n📊 TRANSITION LOG:")
print(f"INPUT:  {before['total_reads']:,} reads")
print(f"OUTPUT: {after['total_reads']:,} reads")
print(f"CHANGE: {before['total_reads'] - after['total_reads']:,} reads removed")
print(f"SURVIVAL RATE: {after['total_reads']/before['total_reads']*100:.1f}%")
print(f"REASON: Low quality, short reads, adapters")

## Step 3: Assembly (MEGAHIT)

In [ ]:
# Install MEGAHIT
!wget -q https://github.com/voutcn/megahit/releases/download/v1.2.9/MEGAHIT-1.2.9-Linux-x86_64-static.tar.gz
!tar -xzf MEGAHIT-1.2.9-Linux-x86_64-static.tar.gz
!cp MEGAHIT-1.2.9-Linux-x86_64-static/bin/megahit* /usr/local/bin/

print("Running MEGAHIT assembly (this takes 1-2 hours)...")
print("Parameters:")
print("  --min-contig-len 500: Only output contigs >= 500bp")
print("  --k-min 21 --k-max 141: K-mer range for assembly")
print()

# Run MEGAHIT
cmd = f"""megahit \
    -1 clean_1.fastq.gz \
    -2 clean_2.fastq.gz \
    -o assembly \
    --min-contig-len 500 \
    --k-min 21 \
    --k-max 141 \
    --k-step 12 \
    -t {THREADS} \
    -m 0.9"""

!{cmd}
log_command(cmd)

In [ ]:
# Calculate assembly statistics
seqs = []
with open('assembly/final.contigs.fa') as f:
    s = ''
    for line in f:
        if line.startswith('>'):
            if s: seqs.append(len(s))
            s = ''
        else: s += line.strip()
    if s: seqs.append(len(s))

seqs.sort(reverse=True)
total = sum(seqs)

# N50
n50_sum = 0
n50 = 0
for l in seqs:
    n50_sum += l
    if n50_sum >= total/2:
        n50 = l
        break

print("="*60)
print("STEP 3 COMPLETE: Assembly")
print("="*60)
print(f"\n📊 TRANSITION LOG:")
print(f"INPUT:  Clean FASTQ reads")
print(f"OUTPUT: {len(seqs):,} contigs")
print(f"TOTAL SIZE: {total:,} bp")
print(f"N50: {n50:,} bp")
print(f"LARGEST: {seqs[0]:,} bp")

# Save to Drive
!cp -r assembly {BASE_DIR}/intermediate/

## Step 4: ORF Prediction (Prodigal)

In [ ]:
# Install Prodigal
!apt-get install -qq prodigal

print("Running Prodigal ORF prediction...")
print("Parameters:")
print("  -p meta: Metagenomic mode (required for mixed organisms)")
print()

# Run Prodigal in metagenomic mode
cmd = """prodigal \
    -i assembly/final.contigs.fa \
    -a proteins.faa \
    -d genes.fna \
    -o genes.gff \
    -f gff \
    -p meta \
    -q"""

!{cmd}
log_command(cmd)

# Count ORFs
total_orfs = !grep -c ">" proteins.faa
total_orfs = int(total_orfs[0])

# Count complete vs partial
complete = !grep -c "partial=00" proteins.faa
complete = int(complete[0])

print("="*60)
print("STEP 4 COMPLETE: ORF Prediction")
print("="*60)
print(f"\n📊 TRANSITION LOG:")
print(f"INPUT:  {len(seqs):,} contigs")
print(f"OUTPUT: {total_orfs:,} proteins predicted")
print(f"COMPLETE ORFs: {complete:,}")
print(f"PARTIAL ORFs: {total_orfs - complete:,}")

# Save to Drive
!mkdir -p {BASE_DIR}/intermediate/orfs
!cp proteins.faa genes.fna genes.gff {BASE_DIR}/intermediate/orfs/

## Step 5A: Build UniRef90 Database (Using Your File)

In [ ]:
# Install DIAMOND
!wget -q https://github.com/bbuchfink/diamond/releases/download/v2.1.8/diamond-linux64.tar.gz
!tar -xzf diamond-linux64.tar.gz
!mv diamond /usr/local/bin/

# Check if database already exists
UNIREF90_DB = f'{BASE_DIR}/uniref90.dmnd'

if os.path.exists(UNIREF90_DB):
    print(f"✅ Found existing UniRef90 database!")
    print(f"   Copying to working directory...")
    !cp "{UNIREF90_DB}" uniref90.dmnd
else:
    print("Building UniRef90 database from your uniref90.fasta.gz...")
    print("This will take ~15-20 minutes.")
    print()
    !diamond makedb --in "{UNIREF90_FASTA}" -d uniref90 --threads {THREADS}
    # Save for future use
    !cp uniref90.dmnd "{BASE_DIR}/"
    print(f"\n✅ Database saved to {BASE_DIR}/uniref90.dmnd")

!ls -lh uniref90.dmnd

## Step 5B: DIAMOND Search vs UniRef90

In [ ]:
print("Running DIAMOND search against UniRef90...")
print("Parameters:")
print("  --id 25: Minimum 25% identity (twilight zone threshold)")
print("  --evalue 1e-5: Statistical significance cutoff")
print("  --sensitive: More thorough search")
print()
print("This will take 2-3 hours...")

cmd = """diamond blastp \
    -q proteins.faa \
    -d uniref90 \
    -o uniref90_hits.m8 \
    --id 25 \
    --evalue 1e-5 \
    --sensitive \
    --threads 4 \
    --max-target-seqs 1"""

!{cmd}
log_command(cmd)

uniref_hits = !cut -f1 uniref90_hits.m8 | sort -u | wc -l
uniref_hits = int(uniref_hits[0])

print("\n" + "="*60)
print("STEP 5B COMPLETE: UniRef90 Search")
print("="*60)
print(f"\n📊 TRANSITION LOG:")
print(f"INPUT:  {total_orfs:,} proteins")
print(f"OUTPUT: {uniref_hits:,} proteins with UniRef90 hits")
print(f"NO HITS: {total_orfs - uniref_hits:,} proteins (potential novel)")

## Step 5C: SwissProt Search

In [ ]:
# Download and build SwissProt database
print("Downloading SwissProt database (~90 MB)...")
!wget -q https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz
!gunzip -f swissprot.gz
!diamond makedb --in swissprot -d swissprot --quiet

# Search
print("Searching against SwissProt...")
cmd = """diamond blastp \
    -q proteins.faa \
    -d swissprot \
    -o swissprot_hits.m8 \
    --id 25 \
    --evalue 1e-5 \
    --sensitive \
    --threads 4"""

!{cmd}
log_command(cmd)

swiss_hits = !cut -f1 swissprot_hits.m8 | sort -u | wc -l
swiss_hits = int(swiss_hits[0])

print(f"\n📊 SwissProt hits: {swiss_hits:,}")

## Step 5D: Human Contamination Check

In [ ]:
# Download human proteome
print("Downloading human proteome (~15 MB)...")
!wget -q https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000005640/UP000005640_9606.fasta.gz
!gunzip -f UP000005640_9606.fasta.gz
!diamond makedb --in UP000005640_9606.fasta -d human --quiet

# Search with relaxed thresholds
print("Checking for human contamination...")
cmd = """diamond blastp \
    -q proteins.faa \
    -d human \
    -o human_hits.m8 \
    --id 50 \
    --evalue 1e-10 \
    --threads 4"""

!{cmd}
log_command(cmd)

human_hits_count = !cut -f1 human_hits.m8 | sort -u | wc -l
human_hits_count = int(human_hits_count[0])

print(f"\n📊 Human contamination: {human_hits_count} proteins")
if human_hits_count == 0:
    print("✅ PASS - No human contamination detected")
else:
    print("⚠️ WARNING - Human contamination detected!")

## Step 5E: Chimera Detection (VSEARCH)

In [ ]:
# Install VSEARCH
!wget -q https://github.com/torognes/vsearch/releases/download/v2.22.1/vsearch-2.22.1-linux-x86_64.tar.gz
!tar -xzf vsearch-2.22.1-linux-x86_64.tar.gz
!cp vsearch-2.22.1-linux-x86_64/bin/vsearch /usr/local/bin/

print("Running chimera detection...")
cmd = """vsearch \
    --uchime_denovo genes.fna \
    --chimeras chimeras.fna \
    --nonchimeras clean_genes.fna \
    --quiet"""

!{cmd}
log_command(cmd)

chimera_count = !grep -c ">" chimeras.fna 2>/dev/null || echo "0"
chimera_count = int(chimera_count[0])

print(f"\n📊 Potential chimeras detected: {chimera_count}")

## Step 6: Extract Novel Proteins

In [ ]:
# Get all protein IDs
all_proteins = set()
with open('proteins.faa') as f:
    for line in f:
        if line.startswith('>'):
            all_proteins.add(line[1:].split()[0])

# Get proteins WITH hits
def get_hits(filename):
    hits = set()
    try:
        with open(filename) as f:
            for line in f:
                hits.add(line.split()[0])
    except FileNotFoundError:
        pass
    return hits

uniref90_hits = get_hits('uniref90_hits.m8')
swissprot_hits = get_hits('swissprot_hits.m8')
human_hits = get_hits('human_hits.m8')

# Novel = no hits anywhere
all_hits = uniref90_hits | swissprot_hits | human_hits
novel = all_proteins - all_hits

print("="*60)
print("STEP 6: EXTRACT NOVEL PROTEINS")
print("="*60)
print(f"\n📊 TRANSITION LOG:")
print(f"Total proteins: {len(all_proteins):,}")
print(f"UniRef90 hits: {len(uniref90_hits):,}")
print(f"SwissProt hits: {len(swissprot_hits):,}")
print(f"Human hits: {len(human_hits):,}")
print(f"\n🔬 NOVEL (no hits in any database): {len(novel):,}")

# Extract novel sequences
seqs = {}
with open('proteins.faa') as f:
    h = None
    for line in f:
        if line.startswith('>'):
            h = line[1:].split()[0]
            seqs[h] = line
        else:
            seqs[h] += line

with open('novel_raw.faa', 'w') as f:
    for pid in novel:
        f.write(seqs[pid])

log_command(f"Extracted {len(novel)} novel proteins")

## Step 7: Quality Filtering

In [ ]:
# Load novel proteins
novels = []
with open('novel_raw.faa') as f:
    h, s = None, ''
    for line in f:
        if line.startswith('>'):
            if h: novels.append((h, s))
            h, s = line[1:].strip(), ''
        else:
            s += line.strip()
    if h: novels.append((h, s))

# Apply filters
filtered = []
rejected = {'short': 0, 'no_start': 0, 'internal_stop': 0, 'partial': 0}

for header, seq in novels:
    if len(seq) < 100:
        rejected['short'] += 1
    elif not seq.startswith('M'):
        rejected['no_start'] += 1
    elif '*' in seq[:-1]:
        rejected['internal_stop'] += 1
    elif 'partial=10' in header or 'partial=01' in header or 'partial=11' in header:
        rejected['partial'] += 1
    else:
        filtered.append((header, seq))

print("="*60)
print("STEP 7: QUALITY FILTERING")
print("="*60)
print(f"\n📊 TRANSITION LOG:")
print(f"INPUT:  {len(novels):,} novel proteins")
print(f"OUTPUT: {len(filtered):,} quality-filtered proteins")
print(f"\nREJECTED:")
for reason, count in sorted(rejected.items(), key=lambda x: -x[1]):
    print(f"  {reason}: {count:,}")

# Save filtered
with open(f'{BASE_DIR}/output/truly_novel.faa', 'w') as f:
    for h, s in filtered:
        f.write(f'>{h}\n{s}\n')

log_command(f"Quality filtered: {len(novels)} -> {len(filtered)}")
print(f"\n✅ Saved to: {BASE_DIR}/output/truly_novel.faa")

## Step 8A: Amino Acid Composition Check

In [ ]:
from collections import Counter

EXPECTED = {'A': 0.083, 'R': 0.055, 'N': 0.041, 'D': 0.055, 'C': 0.014,
            'Q': 0.039, 'E': 0.068, 'G': 0.071, 'H': 0.023, 'I': 0.060,
            'L': 0.097, 'K': 0.058, 'M': 0.024, 'F': 0.039, 'P': 0.047,
            'S': 0.066, 'T': 0.053, 'W': 0.011, 'Y': 0.029, 'V': 0.069}

all_aa = ''.join(s for h, s in filtered)
counts = Counter(all_aa)
total = sum(counts[aa] for aa in EXPECTED.keys() if aa in counts)

chi_sq = 0
for aa, exp in EXPECTED.items():
    obs = counts.get(aa, 0) / total if total > 0 else 0
    chi_sq += ((obs - exp) ** 2) / exp

print("="*60)
print("VERIFICATION: Amino Acid Composition")
print("="*60)
print(f"Chi-squared: {chi_sq:.4f}")
print("\nInterpretation:")
print("  < 0.3 = Normal protein composition")
print("  > 0.5 = Suspicious (possible artifacts)")
verdict_aa = "PASS" if chi_sq < 0.3 else "FAIL"
print(f"\n📊 VERDICT: {verdict_aa}")

## Step 8B: Protein Classification

In [ ]:
def classify(seq):
    issues = []
    for aa in 'AGLPQRS':
        if aa * 5 in seq: issues.append('repetitive')
    if sum(1 for a in seq if a in 'AILMFWV') / len(seq) > 0.6:
        issues.append('too_hydrophobic')
    if sum(1 for a in seq if a in 'DEKR') / len(seq) > 0.4:
        issues.append('too_charged')
    return 'likely_real' if len(issues) == 0 else 'possibly_artifact'

real, artifact = 0, 0
for h, s in filtered:
    if classify(s) == 'likely_real':
        real += 1
    else:
        artifact += 1

print("="*60)
print("VERIFICATION: Protein Classification")
print("="*60)
print(f"Likely real: {real} ({real/len(filtered)*100:.1f}%)")
print(f"Possibly artifact: {artifact}")
verdict_class = "PASS" if real/len(filtered) > 0.9 else "REVIEW"
print(f"\n📊 VERDICT: {verdict_class}")

## Step 8C: Genomic Context Analysis

In [ ]:
import re
from collections import defaultdict

# Parse GFF to get ORF positions
contig_orfs = defaultdict(list)
with open('genes.gff') as f:
    for line in f:
        if line.startswith('#') or not line.strip():
            continue
        parts = line.strip().split('\t')
        if len(parts) >= 9 and parts[2] == 'CDS':
            contig = parts[0]
            start, end = int(parts[3]), int(parts[4])
            pid_match = re.search(r'ID=([^;]+)', parts[8])
            if pid_match:
                pid = pid_match.group(1)
                contig_orfs[contig].append((start, end, pid))

# Find isolated ORFs
novel_ids = set(h.split()[0] for h, s in filtered)
isolated_novels = []

for contig, orfs in contig_orfs.items():
    for start, end, pid in orfs:
        if pid in novel_ids:
            if len(orfs) == 1:
                isolated_novels.append((pid, 'only_orf_on_contig'))
            elif max(e for s, e, p in orfs) - min(s for s, e, p in orfs) < 500:
                isolated_novels.append((pid, 'very_short_contig'))

print("="*60)
print("VERIFICATION: Genomic Context")
print("="*60)
print(f"Isolated ORFs (lower confidence): {len(isolated_novels)}")
print(f"Contextual ORFs (higher confidence): {len(novel_ids) - len(isolated_novels)}")
verdict_context = "PASS" if len(isolated_novels) < len(novel_ids) * 0.3 else "REVIEW"
print(f"\n📊 VERDICT: {verdict_context}")

## Step 8D: Generate Final Report

In [ ]:
# Classification
classifications = {
    'type_a_completely_novel': [],
    'type_b_structure_known': [],
    'type_c_remote_homolog': [],
    'type_d_domain_hybrid': [],
    'type_e_artifact': []
}

# Add artifacts
artifact_ids = set()
for pid, reason in isolated_novels:
    classifications['type_e_artifact'].append((pid, reason))
    artifact_ids.add(pid)

# Remaining as Type A (preliminary)
for h, s in filtered:
    pid = h.split()[0]
    if pid not in artifact_ids:
        classifications['type_a_completely_novel'].append(pid)

# Generate report
report = f"""# Verification Report - Silent Hunter v6.0

Generated: {datetime.datetime.now().isoformat()}

## FAHDYCONX Transition Summary

| Step | Input | Output | Change |
|------|-------|--------|--------|
| Raw Data | SRR6356483 | FASTQ | Converted |
| QC | {before['total_reads']:,} reads | {after['total_reads']:,} reads | {before['total_reads']-after['total_reads']:,} removed |
| Assembly | Clean reads | {len(seqs):,} contigs | N50={n50:,}bp |
| ORF Prediction | Contigs | {total_orfs:,} proteins | - |
| UniRef90 | {total_orfs:,} proteins | {len(uniref90_hits):,} hits | {total_orfs-len(uniref90_hits):,} no match |
| Quality Filter | {len(novels):,} novel | {len(filtered):,} filtered | {len(novels)-len(filtered):,} rejected |

## Final Counts

| Metric | Value |
|--------|-------|
| Total proteins predicted | {len(all_proteins):,} |
| Novel (no database hits) | {len(novel):,} |
| Quality filtered | {len(filtered):,} |

## Verification Tests

| Test | Result | Verdict |
|------|--------|--------|
| Human contamination | {human_hits_count} matches | {"PASS" if human_hits_count == 0 else "FAIL"} |
| AA composition | Chi-sq = {chi_sq:.3f} | {verdict_aa} |
| Protein classification | {real/len(filtered)*100:.1f}% real | {verdict_class} |
| Genomic context | {len(isolated_novels)} isolated | {verdict_context} |

## Novel Protein Classification (Preliminary)

| Type | Description | Count |
|------|-------------|-------|
| Type A | Completely novel | {len(classifications['type_a_completely_novel'])} |
| Type B | Structure-known | TBD (run Foldseek) |
| Type C | Remote homolog | TBD (run HHblits) |
| Type D | Domain hybrid | TBD (run Pfam) |
| Type E | Possible artifact | {len(classifications['type_e_artifact'])} |

## Conclusion

**{len(filtered):,} novel protein candidates** identified from ISS metagenome.
"""

with open(f'{BASE_DIR}/output/VERIFICATION_REPORT.md', 'w') as f:
    f.write(report)

print(report)
print(f"\n✅ Report saved to: {BASE_DIR}/output/VERIFICATION_REPORT.md")

## Save All Results

In [ ]:
# Save all intermediate files to Drive
!mkdir -p {BASE_DIR}/intermediate/diamond
!cp *_hits.m8 {BASE_DIR}/intermediate/diamond/ 2>/dev/null || true

# Calculate final checksums
!md5sum {BASE_DIR}/output/truly_novel.faa >> {BASE_DIR}/audit/checksums.md5

print("\n" + "="*60)
print("PIPELINE COMPLETE")
print("="*60)
print(f"\nAll files saved to: {BASE_DIR}")
print("\nFinal output:")
!ls -lh {BASE_DIR}/output/

print("\n📋 Next Steps:")
print("1. Review VERIFICATION_REPORT.md")
print("2. Run manual HHblits verification (20 samples)")
print("3. Run manual Foldseek verification (10 samples)")
print("4. Complete NCBI nr spot check")